### DeAp Dispatching P1

Overloading je když máme funkce s více jmény a program si podle rozdílného podpisu - parametry apod. je schopný rozoznat kterou potřebuje. V Pythonu overloading není, nejedná se o statický jazyk. Python má single dispatch a multi dispatch.

In [415]:
from html import escape #poznám dále

In [416]:
def html_escape(arg):
    return escape(str(arg))
    #prostě jen převe argument do stringu a escapne?
    
def html_int(a):
    return f"{a} (<i>{str(hex(a))}</i>)"
#převede integer ok

def html_real(a):
    return f"{round(a,2):.2f}"

def html_str(s):
    return html_escape(s).replace("\n", "<br/>\n")

def html_list(l):
    items = (f"<li>{html_escape(item)}</li>"
            for item in l 
            )
    return "<ul>\n" + "\n".join(items) + "\n</ul>"

def html_dict(d):
    items = (f"<li>{k}={v}</li>"
             for k,v in d.items()
            )
    return "<ul>\n" + "\n" .join(items) + "\n</ul>"

#Prostě máme kotel funkcí pro úpravu všemožných formátů, int, stringu, listů, dictionary apod.
#Největší pozor dát asi na list a dict generátory se stringem - tj. poměrně zajímavá část

In [417]:
print(html_str("""this is 
a multi line string
with special characters: 10 < 100"""))

this is <br/>
a multi line string<br/>
with special characters: 10 &lt; 100


In [418]:
print(html_int(255))

255 (<i>0xff</i>)


In [422]:
html_list([1,2,3,4,5])

'<ul>\n<li>1</li>\n<li>2</li>\n<li>3</li>\n<li>4</li>\n<li>5</li>\n</ul>'

In [423]:
html_list([1,2,3,4,5])

'<ul>\n<li>1</li>\n<li>2</li>\n<li>3</li>\n<li>4</li>\n<li>5</li>\n</ul>'

Jasné prostě kotel funkcí co zde máme pro hmtl úpravy.

#### NYNÍ CHCEME NAPSAT FUNKCI - CO URČÍ JAKOU FUNKCI MÁ POUŽÍT

In [424]:
from decimal import Decimal

In [439]:
#budeme to psát zase nahrubo a ručně 
def htmlize(arg):
    #pokud máme integere vrátíme funkci pro integer
    if isinstance(arg, int):
        return html_int(arg)
    elif isinstance(arg, float) or isinstance(arg, Decimal):
        return html_real(arg) #kryjeme decimal a float
    elif isinstance(arg, str):
        return html_str(arg)
    elif isinstance(arg, list) or isinstance(arg, tuple):
        return html_list(arg)
    elif isinstance(arg, dict):
        return html_dict(arg)
    elif isinstance(arg,set):
        return html_set(arg)
    else:
        return html_escape(arg)
#netřeba řešit řešíme zda je něco instance něčeho - tzn. zda náš argument
#je instance něčeho a podle toho navracíme funkci

In [428]:
htmlize(100)

'100 (<i>0x64</i>)'

In [432]:
htmlize([1,2,3,4])

'<ul>\n<li>1</li>\n<li>2</li>\n<li>3</li>\n<li>4</li>\n</ul>'

Tohle je tedy náš hardcoded dispatcher.

In [433]:
print(htmlize([1,2,3,4]))

<ul>
<li>1</li>
<li>2</li>
<li>3</li>
<li>4</li>
</ul>


Samozřejmě to není dokonalé. Jakmile bude typ v typu už máme tak trochu problém. Nenajde to správnou úpravu.

In [434]:
def func1():
    func2()

In [435]:
def func2():
    print("func2")

Pokud nevolám funkci 1 dokud nemám nadefinovanou func2 - není problém a po nadefinování mohu vše volat. V podstatě není problém psát funkci, která není ještě definovaná - pokud jí dodefinujeme před spuštěním.

In [438]:
def html_escape(arg):
    return escape(str(arg))
    #prostě jen převe argument do stringu a escapne?
    
def html_int(a):
    return f"{a} (<i>{str(hex(a))}</i>)"
#převede integer ok

def html_real(a):
    return f"{round(a,2):.2f}"

def html_str(s):
    return html_escape(s).replace("\n", "<br/>\n")

def html_list(l):
    items = (f"<li>{htmlize(item)}</li>"
            for item in l 
            )
    return "<ul>\n" + "\n".join(items) + "\n</ul>"

def html_dict(d):
    items = (f"<li>{html_escape(k)}={htmlize(v)}</li>"
             for k,v in d.items()
            )
    return "<ul>\n" + "\n" .join(items) + "\n</ul>"

def html_set(arg):
    return html_list(arg)

A tudíž mohu takto upravit naší funkci - která vybírá sama sebe ? :D, viz dict a list

In [437]:
print(htmlize(["""
Python rocks! 0 < 1
""", (10,20,30),100]))

<ul>
<li><br/>
Python rocks! 0 &lt; 1<br/>
</li>
<li><ul>
<li>10 (<i>0xa</i>)</li>
<li>20 (<i>0x14</i>)</li>
<li>30 (<i>0x1e</i>)</li>
</ul></li>
<li>100 (<i>0x64</i>)</li>
</ul>


Jo takže vidíme že pozná i v jedné části co je to za část uvnitř :D. Takže například použije htmlize na list - ale uvnitr 10,20,30 použije htmlize na integer

#### Nyní to zkusíme předělat trochu chytřeji

In [445]:
def htmlize(arg):
    #nejdříve si uděláme registr typ: funkce
    registry = {
        object: html_escape,
        int: hmtl_int,
        float: hmtl_int,
        Decimal: html_int,
        str: html_str,
        list: html_list,
        tuple: html_list,
        set: html_set,
        dict: html_dict        
    }
    #registry object je defaultní hodnota - prostě jen escape
    fn = registry.get(type(arg), registry[object])
    
    return fn(arg)
#získáme typ argumentu a rovnou získáme název funkce
#poté pouze spustíme funkce s argumentem
#pokud mu totiž zadám například "ahoj" - tak registry.get - type
#získá typ - str - a je to jako kdybych napsal ..
#najdi mi hodnotu k str - a tudíž najde funkci html_str

In [452]:
registry2 = {
    object: html_escape,
    str: html_str,
    list: html_list,
    tuple: html_list,
    set: html_set,
    dict: html_dict        
    }

In [457]:
registry2.get(type("ahoj"))

<function __main__.html_str(s)>

In [455]:
type("ahoj")

str

Zde pouze k dopochopení. Obecně klasika, pokud mám MASS ELIF - je lepší použít dictionary

### DeAp Dispatching P2

Máme hardcoded registr, který nechceme. Potřebujeme to mít trochu vyladěnější.

In [458]:
def singledispatch(fn):
    registry = {}
    
    registry[object] = fn #defaultní funkce?
    
    def inner(arg):
        return registry[object](arg)
    
    return inner

In [459]:
@singledispatch
def htmlize(a):
    return escape(str(a))

In [461]:
htmlize("1 < 100")
#když volám htmlize - voláme inner - kvůli dekorátoru
# a inner má navrátit to co je v registry - ale to je funkce kterou pouštíme
#takže spustí return escape(str(a))
#tzn v podstatě se otočí dokola ale ve výsledku zatím nic neudělá

'1 &lt; 100'

In [463]:
escape(str("1 < 100")) #to samé, ale projede kolečko :)

'1 &lt; 100'

In [467]:
def singledispatch(fn):
    registry = {}
    
    registry[object] = fn #defaultní funkce
    registry[int] = lambda a: f"{a} (<i>{str(hex(a))}</i>)"
    registry[str] = lambda s: escape(s).replace("\n", "<br/>\n")
    
    
    def inner(arg):
        return registry.get(type(arg), registry[object])(arg) #(arg) - call funce na arg
    
    return inner
#v podstatě mám decorátor - který projede inner funkci, a inner funkce
#spustí funkci kterou najde v registrech

In [466]:
@singledispatch
def htmlize(a):
    return escape(str(a))

In [471]:
htmlize(100)

<function __main__.singledispatch.<locals>.<lambda>(a)>

Podle všeho nic extra neřešíme. Náš registry postup tedy nechceme. Potřebujeme funkci která nám umožní vkládat klíče a hodnoty do registru. Budeme psát decorátor unvitř decoratoru.

In [519]:
def singledispatch(fn):
    registry = {}
    
    registry[object] = fn #defaultní funkce
    
    def decorator(arg):
        return registry.get(type(arg), registry[object])(arg) #(arg) - call funce na arg
    
    def register(type_): #naše dec.factory
        def inner(fn):
            registry[type_] = fn
            return fn
        #měla by přiřadit funkci k typu v registru
        return inner
    
    def dispatch(type_):
        return resgitry.get(type_, registry[object])
    
    decorator.dispatch = dispatch #jiný způsob pro registr
    decorator.register = register #handle na decorator
    decorator.registry = registry #abych viděl registry
    #teď v podstatě přiřadíme propojení našeho decorátoru na funkci register
    #abychom jí mohli použít
    return decorator

In [520]:
@singledispatch
def htmlize(a):
    return escape(str(a))

In [521]:
htmlize("tuduu") #použije prostě defaultní hodnotu

'tuduu'

In [522]:
htmlize(100) #nic to nedělá že náš integer není v registrech

'100'

Jenže jak použijeme naší registre funkci uvnitř decorátoru abychom mohli evidovat typy?

Přidali jsme atribut - decorated.register - abychom měli páku.

In [513]:
#spouštím decorátor v decorátoru (singledispatch)
#náš decorátor v decorátoru má jako parametr typ
#a jako funkci očekává funkci - kterou zaeviduje do registrů
@htmlize.register(int)
def html_int(a):
    return f"{a} (<i>{str(hex(a))}</i>)"
#navíc já beru htmlize.register což je psycho zápis již deokoravné funkce výše

In [514]:
htmlize(100)

'100 (<i>0x64</i>)'

A nyní to použije správnou funkci, protože to použije námi dodaný kod.

In [515]:
@htmlize.register(list)
def html_list(l):
    items = (f"<li>{htmlize(item)}</li>"
        for item in l 
        )
    return "<ul>\n" + "\n".join(items) + "\n</ul>"

In [516]:
html_list = htmlize.register(list)(html_list) #bleeh nedekorovaný zápis

Navíc můžu stackovat dekorátory takto

In [517]:
@htmlize.register(list)
@htmlize.register(tuple)
def html_list(l):
    items = (f"<li>{htmlize(item)}</li>"
        for item in l 
        )
    return "<ul>\n" + "\n".join(items) + "\n</ul>"
#takto přidá stejnou funkci k tuplu i listu

Nyní bychom ale rádi viděli jak registor vypadá

In [518]:
htmlize.registry

{object: <function __main__.htmlize(a)>,
 int: <function __main__.html_int(a)>,
 list: <function __main__.html_list(l)>,
 tuple: <function __main__.html_list(l)>}

A nyní vidím, že máme krásně přidané naše registry funkcí.

In [524]:
htmlize.dispatch

<function __main__.singledispatch.<locals>.dispatch(type_)>

#### A nyní PROČ to co píšeme je SHITTY

In [526]:
from numbers import Integral

In [528]:
class Person:
    pass

class Student(Person):
    pass

In [529]:
p = Student()

In [530]:
type(p)

__main__.Student

Ok dvě classy, typ p je student jasné. Ale student je i hodnota - typ to ale nepodchytí.

In [531]:
isinstance(p, Student), isinstance(p,Person)

(True, True)

Ale isinstance pozná vše správně.

In [532]:
@singledispatch
def htmlize(a):
    return escape(str(a))

In [533]:
@htmlize.register(Integral)
def html_integral_number(a):
    return f"{a} (<i>{str(hex(a))}</i>)"

In [534]:
isinstance(10, Integral)

True

In [535]:
htmlize(10)

'10'

Protože kouká na typ, ale nekouká na isinstance. Musel bych naházet za sebe více deokorátorů s hodnotami, není to tolik "generic".

In [536]:
from collections.abc import Sequence

In [537]:
isinstance([1,2,3],Sequence)

True

Mám tedy abstraktní typy které pod sebe sdružují více typů, a ty můžeme dát do dekorátoru. Type to ale stejně neřeší.

### DeAp Dispatching P3

In [539]:
from functools import singledispatch

In [541]:
from numbers import Integral
from collections.abc import Sequence

V podstatě functools má to o co jsme se snažili zabudovaný v sobě

In [543]:
@singledispatch
def htmlize(a):
    return escape(str(a))

In [544]:
htmlize.registry #disponuje registry

mappingproxy({object: <function __main__.htmlize(a)>})

In [545]:
@htmlize.register(Integral)
def htmlize_intgral_number(a):
    return f"{a} (<i>{str(hex(a))}</i>)"

Stejně jako u naší verze můžeme vkládat.

In [546]:
htmlize.registry

mappingproxy({object: <function __main__.htmlize(a)>,
              numbers.Integral: <function __main__.htmlize_intgral_number(a)>})

In [547]:
htmlize.dispatch(int)

<function __main__.htmlize_intgral_number(a)>

A to co u nás nefungovalo, zde funguje :). 

In [550]:
isinstance(10,int), isinstance(10, Integral), isinstance(True, Integral)

(True, True, True)

Pouze pro ukázku že Integral je více typů. Konec nejdelšího a asi nejnáročnějšího úseku celé kapitoly P1.